# Dognition User Activity and Profile Completion

Dognition provides dog owners with insight into how their pet's mind works using a series of 20 games that test different aspects of dog behavior. The results of the games are analyzed to determine which of six personality types describes the dog. In addition, a unique profile report looks in-depth at the dog's cognitive style. The data used here comes from a Coursera MOOC, Managing Big Data with MySQL, by Duke University. This notebook builds on work done in that course. The queries below look at user engagement and attrition. For more information on Dognition, visit www.dognition.com.

### Time period covered by game data 

In [21]:
%%sql

SELECT 
    MIN(created_at) AS Start, 
    MAX(created_at) AS End, 
    TIMESTAMPDIFF(DAY, MIN(created_at), MAX(created_at)) + 1 AS DaysOfData
FROM
    complete_tests; 

1 rows affected.


Start,End,DaysOfData
2013-02-05 18:26:54,2015-10-12 22:20:32,980


### Total number of individual dog records  
List of dog ids and user ids where neither the dog nor the user is marked for exclusion from analysis.

In [22]:
%%sql

SELECT
        DISTINCT(d.dog_guid) AS DogID, 
        d.user_guid AS UserID
FROM 
        dogs d
            LEFT JOIN users u 
                ON d.user_guid = u.user_guid
WHERE 
        ((d.exclude = 0 OR d.exclude IS NULL) 
             AND 
        (u.exclude = 0 OR u.exclude IS NULL)
             AND 
        u.user_guid IS NOT NULL)
LIMIT 10;  

10 rows affected.


DogID,UserID
fd27b272-7144-11e5-ba71-058fbc01cf0b,ce134e42-7144-11e5-ba71-058fbc01cf0b
fd417cac-7144-11e5-ba71-058fbc01cf0b,ce134e42-7144-11e5-ba71-058fbc01cf0b
fd27b5ba-7144-11e5-ba71-058fbc01cf0b,ce1353d8-7144-11e5-ba71-058fbc01cf0b
fd3fb0f2-7144-11e5-ba71-058fbc01cf0b,ce1353d8-7144-11e5-ba71-058fbc01cf0b
fd27b6b4-7144-11e5-ba71-058fbc01cf0b,ce135ab8-7144-11e5-ba71-058fbc01cf0b
fd27b79a-7144-11e5-ba71-058fbc01cf0b,ce13507c-7144-11e5-ba71-058fbc01cf0b
fd27b948-7144-11e5-ba71-058fbc01cf0b,ce13615c-7144-11e5-ba71-058fbc01cf0b
fd27bbbe-7144-11e5-ba71-058fbc01cf0b,ce135f2c-7144-11e5-ba71-058fbc01cf0b
fd27c1c2-7144-11e5-ba71-058fbc01cf0b,ce136a1c-7144-11e5-ba71-058fbc01cf0b
fd27c0fa-7144-11e5-ba71-058fbc01cf0b,ce136a1c-7144-11e5-ba71-058fbc01cf0b


### Number of dogs completing each assessment game
How many dogs continue to each subsequent game and set of games? Do any games stand out as a dropping-out point?   

In [19]:
%%sql

SELECT TestNames.test_name AS Test_Name, TestNames.subcategory_name AS SubCat, COUNT(TestNames.test_name) AS TestCt
FROM
(SELECT 
        DISTINCT(d.dog_guid) AS dogid, 
        d.user_guid AS User, 
        c.subcategory_name, 
        c.test_name, 
        d.dimension, 
        d.created_at 
FROM 
        dogs d
            LEFT JOIN users u 
                ON d.user_guid = u.user_guid
            LEFT JOIN complete_tests c 
                ON c.dog_guid = d.dog_guid
WHERE 
        ((d.exclude = 0 OR d.exclude IS NULL) 
             AND 
        (u.exclude = 0 OR u.exclude IS NULL)
             AND
        u.user_guid IS NOT NULL
             AND
        c.subcategory_name IN ("Empathy","Communication","Cunning","Memory","Reasoning"))) AS TestNames
GROUP BY Test_Name
ORDER BY TestCt DESC;

20 rows affected.


Test_Name,SubCat,TestCt
Yawn Warm-up,Empathy,16481
Yawn Game,Empathy,15418
Eye Contact Warm-up,Empathy,14096
Eye Contact Game,Empathy,13767
Treat Warm-up,Communication,9986
Arm Pointing,Communication,9645
Foot Pointing,Communication,9020
Watching,Cunning,6781
Turn Your Back,Cunning,6678
Cover Your Eyes,Cunning,6527


### Registered only
How many dogs that were registered didn't complete any activities? 

In [14]:
%%sql

SELECT COUNT(TestNames.dogid) AS Registered_Only
FROM
(SELECT 
        DISTINCT(d.dog_guid) AS dogid, 
        d.user_guid AS User, 
        c.subcategory_name, 
        c.test_name, 
        d.dimension, 
        d.created_at 
FROM 
        dogs d
            LEFT JOIN users u 
                ON d.user_guid = u.user_guid
            LEFT JOIN complete_tests c 
                ON c.dog_guid = d.dog_guid
WHERE 
        (d.exclude = 0 OR d.exclude IS NULL) 
             AND 
        (u.exclude = 0 OR u.exclude IS NULL)
             AND
        u.user_guid IS NOT NULL
             AND
        c.test_name IS NULL) AS TestNames;

1 rows affected.


Registered_Only
16963


### Total dogs given a personality profile
How many dogs complete enough games to create a profile? 

In [15]:
%%sql

SELECT 
       COUNT(tc.test_name) AS Completers
FROM
       (SELECT 
             DISTINCT(c.dog_guid), 
             d.user_guid, 
             d.dimension,
             c.test_name
       FROM dogs d
             LEFT JOIN users u 
                ON d.user_guid = u.user_guid
             LEFT JOIN complete_tests c 
                ON c.dog_guid = d.dog_guid     
       WHERE 
             ((d.exclude = 0 OR d.exclude IS NULL) 
                AND 
             (u.exclude = 0 OR u.exclude IS NULL))  
                AND
             u.user_guid IS NOT NULL
                AND
             (d.dimension IS NOT NULL AND dimension <> "")
       GROUP BY 
             c.dog_guid) AS tc;
   

1 rows affected.


Completers
3743


### Completion time for assessment games
Beginning with the first test, how quickly are those who finish completing all the games? 

In [20]:
%%sql

SELECT 
    Completion.TimeToFinish, 
    COUNT(Completion.TimeToFinish) AS CompletionTime
FROM 
    (SELECT 
            Start.DogID,
            Start.Start_Time,
            mEndTime.EndTime AS End_Time,
            DATEDIFF(mEndTime.EndTime, Start.Start_Time) + 1 AS DaysEngaged,
            CASE 
                WHEN DATEDIFF(mEndTime.EndTime, Start.Start_Time) + 1 = 1 THEN "One Day"
                WHEN DATEDIFF(mEndTime.EndTime, Start.Start_Time) + 1 BETWEEN 2 AND 7 THEN "2-7 Days"
                WHEN DATEDIFF(mEndTime.EndTime, Start.Start_Time) + 1 BETWEEN 8 AND 30 THEN "8-30 Days"
                WHEN DATEDIFF(mEndTime.EndTime, Start.Start_Time) + 1 BETWEEN 31 AND 60 THEN "31-60 Days"
                WHEN DATEDIFF(mEndTime.EndTime, Start.Start_Time) + 1 BETWEEN 61 AND 90 THEN "61-90 Days"            
                WHEN DATEDIFF(mEndTime.EndTime, Start.Start_Time) + 1 BETWEEN 91 AND 180 THEN "91-180 Days"
                WHEN DATEDIFF(mEndTime.EndTime, Start.Start_Time) + 1 BETWEEN 181 AND 365 THEN "181-365 Days"            
                WHEN DATEDIFF(mEndTime.EndTime, Start.Start_Time) + 1 > 365 THEN "Over a Year"
            END AS TimeToFinish
                    FROM        
                        (SELECT 
                                DISTINCT(d.dog_guid) AS DogID, 
                                MIN(c.created_at) AS Start_Time
                         FROM 
                                dogs d
                                    LEFT JOIN users u 
                                        ON d.user_guid = u.user_guid
                                    LEFT JOIN complete_tests c 
                                        ON c.dog_guid = d.dog_guid
                         WHERE 
                                ((d.exclude = 0 OR d.exclude IS NULL) 
                                   AND 
                                (u.exclude = 0 OR u.exclude IS NULL)
                                   AND
                                u.user_guid IS NOT NULL
                                   AND
                                d.dimension IS NOT NULL AND d.dimension <> "")
                         GROUP BY 
                                DogID
                         HAVING      
                                COUNT(c.test_name) >= 19) AS Start
                                LEFT JOIN 
                                    (SELECT
                                        DISTINCT(c.dog_guid) AS mDogID, 
                                        MAX(c.created_at) AS EndTime
                                     FROM complete_tests c
                                     GROUP BY c.dog_guid) AS mEndTime
                                ON 
                                         (mEndTime.mDogID = Start.DogID)
                                WHERE 
                                         DATEDIFF(mEndTime.EndTime, Start.Start_Time) + 1
                                ORDER BY 
                                         DaysEngaged DESC) AS Completion
                        GROUP BY 
                                         Completion.TimeToFinish
                        ORDER BY 
                                CASE
                                    WHEN Completion.TimeToFinish = "One Day" THEN 1
                                    WHEN Completion.TimeToFinish = "2-7 Days" THEN 2
                                    WHEN Completion.TimeToFinish = "8-30 Days" THEN 3
                                    WHEN Completion.TimeToFinish = "31-60 Days" THEN 4
                                    WHEN Completion.TimeToFinish = "61-90 Days" THEN 5            
                                    WHEN Completion.TimeToFinish = "91-180 Days" THEN 6
                                    WHEN Completion.TimeToFinish = "181-365 Days" THEN 7            
                                    WHEN Completion.TimeToFinish = "Over a Year" THEN 8
                                END;

8 rows affected.


TimeToFinish,NrDogs
One Day,268
2-7 Days,1055
8-30 Days,1128
31-60 Days,392
61-90 Days,202
91-180 Days,274
181-365 Days,310
Over a Year,114


### Completion time statistics 
What are the minimum, maximum and average completion times? Do completion times appear to be normally distributed?  

In [17]:
%%sql

SELECT MIN(UserTime.DaysEngaged) AS Minimum, MAX(UserTime.DaysEngaged) AS Maximum, AVG(UserTime.DaysEngaged) AS Average
FROM
(SELECT 
        Start.DogID,
        Start.Start_Time,
        mEndTime.EndTime AS End_Time,
        DATEDIFF(mEndTime.EndTime, Start.Start_Time) + 1 AS DaysEngaged
FROM        
        (SELECT 
            DISTINCT(d.dog_guid) AS DogID, 
            MIN(c.created_at) AS Start_Time
        FROM 
            dogs d
                LEFT JOIN users u 
                    ON d.user_guid = u.user_guid
                LEFT JOIN complete_tests c 
                    ON c.dog_guid = d.dog_guid
        WHERE 
            ((d.exclude = 0 OR d.exclude IS NULL) 
               AND 
            (u.exclude = 0 OR u.exclude IS NULL)
               AND
            u.user_guid IS NOT NULL
               AND
            d.dimension IS NOT NULL)
        GROUP BY 
                DogID
        HAVING      
                COUNT(c.test_name) >= 20) AS Start
        LEFT JOIN 
            (SELECT
                DISTINCT(c.dog_guid) AS mDogID, 
                MAX(c.created_at) AS EndTime
                FROM complete_tests c
                GROUP BY c.dog_guid) AS mEndTime
        ON (mEndTime.mDogID = Start.DogID)) AS UserTime;

1 rows affected.


Minimum,Maximum,Average
1,961,61.9446
